#Класс перцептрон



In [3]:
import torch
class Value:
    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None # function
        self._prev = set(_children) # set of Value objects
        self._op = _op # the op that produced this node, string ('+', '-', ....)

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(data = self.data+other.data,
                    _children = (self, other),
                    _op = "+")
        def _backward():
            if out.grad == None:
                self.grad = other.grad*out.grad
                other.grad = self.grad*out.grad
            else:
                self.grad+=out.grad
                other.grad+=out.grad
            return self.grad+other.grad
        out._backward = _backward
        return out

    def __radd__(self, other): # other + self
        return self + other
    def __neg__(self): # -self
        return self * -1
    def __sub__(self, other): # self - other
        return self + (-other)
    def __rsub__(self, other): # other - self
        return other + (-self)

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(data = self.data*other.data,
                    _children = (self, other),
                    _op = "*")
        def _backward():
            if out.grad == None:
                self.grad = other.data*out.grad
                other.grad = self.data*out.grad
            else:
                self.grad+=out.grad*other.data
                other.grad+=out.grad*self.data
        out._backward = _backward
        return out

    def __rmul__(self, other): # other * self
        return self * other
    def __truediv__(self, other): # self / other
        return self * other**-1
    def __rtruediv__(self, other): # other / self
        return other * self**-1
    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})"

    def __pow__(self, other):
        assert isinstance(other, (float, int)), "only supporting int/floats power now"
        out = Value(data = self.data**other,
                    _children = (self, self),
                    _op = "**")
        def _backward():
            if out.grad == None:
                self.grad = (self.data ** (other-1))*(other) * out.grad
            else:
              self.grad+= (self.data ** (other-1))*(other) * out.grad
        out._backward = _backward
        return out

    def __relu__(self):
        out = Value(max(self.data, 0),
                    _children = (self, self),
                    _op = "&")
        def _backward():
            if out.grad == None:
                self.grad = min(self.data, 0)
            else:
                self.grad+=min(self.data, 0) * out.grad
        out._backward = _backward
        return out


    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()



In [14]:
import random

class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(0) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin: bool = nonlin

    def __call__(self, x):
        act = sum((self.w[i]*x[i] for i in range(len(self.w))), self.b)
        return max(0, act) if self.nonlin else act

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        self.w = [[random.random() for _ in range(nin)] for _ in range(nout)]
        self.b = [random.random() for _ in range(nout)]

    def __call__(self, x):
        out = [self.neurons[j](x) for j in range(len(self.neurons))]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [self.w[j] for j in range(len(self.neurons))]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin]+nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=(i!=len(nouts)-1)) for i in range(len(nouts))]

    def __call__(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        return out

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        repr_str = '\n'.join(str(layer) for layer in self.layers)
        return f"MLP of [{repr_str}]"

In [15]:
nn = MLP(3, [4, 4, 1])
print(nn)
print("number of parameters", len(nn.parameters()))

MLP of [Layer of [ReLUNeuron(3), ReLUNeuron(3), ReLUNeuron(3), ReLUNeuron(3)]
Layer of [ReLUNeuron(4), ReLUNeuron(4), ReLUNeuron(4), ReLUNeuron(4)]
Layer of [ReLUNeuron(4)]]
number of parameters 9
